In [4]:
import sys, os
sys.path.append(os.path.realpath(".."))
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit"
from sacred.observers import MongoObserver
import pickle as pkl
from addict import Dict
from sklearn.pipeline import Pipeline
import clinical_text_analysis as cta
import pandas as pd
import numpy as np
import random
from os import path
import tensorflow as tf
tf.enable_eager_execution()
import data_reader as read
import util_funcs
import string

from addict import Dict
import sacred
import preprocessingV2.preprocessingV2 as ppv2
from keras_models.metrics import f1, sensitivity, specificity, auroc
from sklearn.metrics import f1_score, roc_auc_score, classification_report


def read_tfrecord(example):
    features = { \
               'data':  tf.io.FixedLenFeature([21*1000], tf.float32,),\
               'label':  tf.io.FixedLenFeature([1], tf.int64,),\
               'subtypeLabel':  tf.io.FixedLenFeature([1], tf.int64,),\
               'session':  tf.io.FixedLenFeature([1], tf.int64,), \
               'montage':  tf.io.FixedLenFeature([22], tf.int64,)}

    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)
#     return example

    data = tf.reshape(example['data'], [21,1000,1])
    # data = (example['data'])

    class_label = tf.cast(example['label'], tf.int32)

    # del example
    return data, class_label


In [17]:

def get_batched_dataset(filenames, batch_size, max_queue_size=10,  n_process=4, is_train=False):
    option_no_order = tf.data.Options()
    option_no_order.experimental_deterministic = False

    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.with_options(option_no_order)
    dataset = dataset.interleave(tf.data.TFRecordDataset, cycle_length=16, num_parallel_calls=n_process)
#     
    dataset = dataset.map(read_tfrecord, num_parallel_calls=n_process)
    
#     dataset = dataset.cache() # IF this dataset fits in RAM
    dataset = dataset.repeat()
    if is_train:
        dataset = dataset.shuffle(256)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    if is_train:
        dataset = dataset.prefetch(max_queue_size)
    else:
        dataset = dataset.prefetch(int(max_queue_size/4)) #store a lot less for the other sets to avoid wasting memory

    return dataset


In [18]:
data = get_batched_dataset(["/n/scratch2/ms994/train_4s.tfr"], 64, is_train=True)

In [7]:
num_train = 0
for record in tf.python_io.tf_record_iterator("/n/scratch2/ms994/train_4s.tfr"):
     num_train += 1
num_valid = 0
for record in tf.python_io.tf_record_iterator("/n/scratch2/ms994/valid_4s.tfr"):
     num_valid += 1
num_test = 0
for record in tf.python_io.tf_record_iterator("/n/scratch2/ms994/test_4s.tfr"):
     num_test += 1

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [24]:
inputLayer = tf.keras.layers.Input((21,1000, 1))
x = inputLayer
x = tf.keras.layers.Dense(2)(x)
model = tf.keras.Model(inputs=inputLayer, outputs=x)

In [25]:
model.compile("adam", loss="categorical_crossentropy")

In [20]:
someData = data.take(10)

In [21]:
caught = []
for i in someData.make_one_shot_iterator():
    caught.append(i)

In [22]:
caught[0]

(<tf.Tensor: id=505, shape=(64, 21, 1000, 1), dtype=float32, numpy=
 array([[[[-1.53863311e-01],
          [-2.52759665e-01],
          [-2.31898725e-01],
          ...,
          [-7.22482443e-01],
          [-5.40659285e+00],
          [ 4.77428532e+00]],
 
         [[-1.41050720e+01],
          [ 1.05877876e+01],
          [ 5.58325815e+00],
          ...,
          [ 7.36761427e+00],
          [ 1.44125786e+01],
          [-1.37521992e+01]],
 
         [[-1.78998814e+01],
          [-8.27991772e+00],
          [ 3.03435555e+01],
          ...,
          [ 4.66498315e-01],
          [ 6.54012632e+00],
          [ 7.99024391e+00]],
 
         ...,
 
         [[ 4.12863541e+00],
          [-9.75135422e+00],
          [-2.54024196e+00],
          ...,
          [ 9.97674179e+00],
          [-8.31331825e+00],
          [ 1.07679596e+01]],
 
         [[ 2.46525121e+00],
          [ 9.77745533e+00],
          [-1.51566906e+01],
          ...,
          [-6.96288288e-01],
          [-3.180

In [26]:
model.predict(data, steps=10)

array([[[[ 1.41838297e-01,  6.74303528e-03],
         [ 2.73460686e-01,  1.30004026e-02],
         [-3.22763585e-02, -1.53442787e-03],
         ...,
         [ 5.94954252e+00,  2.82843053e-01],
         [-2.15776711e+01, -1.02580905e+00],
         [ 9.29771709e+00,  4.42016244e-01]],

        [[ 2.19767418e+01,  1.04478097e+00],
         [-1.34979151e-02, -6.41694933e-04],
         [-2.31362171e+01, -1.09990275e+00],
         ...,
         [ 1.14845095e+01,  5.45976996e-01],
         [-9.51506615e+00, -4.52349126e-01],
         [-9.02132320e+00, -4.28876430e-01]],

        [[ 1.49202728e+01,  7.09314287e-01],
         [-1.06394262e+01, -5.05801558e-01],
         [ 1.59423971e+00,  7.57906362e-02],
         ...,
         [-2.26511879e+01, -1.07684433e+00],
         [-8.78019905e+00, -4.17413324e-01],
         [-1.19116030e+01, -5.66281140e-01]],

        ...,

        [[-7.25082493e+00, -3.44706386e-01],
         [ 3.78778481e+00,  1.80072427e-01],
         [ 6.12865877e+00,  2.91358292